In [ ]:
pip install tensorflow tensorflow-federated numpy

In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

#Simulate clients with split datasets:
#ex: num_clients = 3 dataset = 9
def create_client_data(num_clients, dataset):
    client_data = []
    #9 / 3 = 3  data_per_client = 3
    data_per_client = len(dataset) // num_clients
    for i in range(num_clients):
      # i = 1, dataset[3:6]
        client_data.append(dataset[i * data_per_client: (i + 1) * data_per_client])
    return client_data

# Load and preprocess the dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), _ = mnist.load_data()
x_train, y_train = x_train / 255.0, y_train

num_clients = 10
client_data = create_client_data(num_clients, list(zip(x_train, y_train)))


ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.11/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def agg_weights(client_weights):
    average_weights = [np.mean([w[i] for w in client_weights], axis=0) for i in range(len(client_weights[0]))]
    return average_weights


In [ ]:
def client_update(model, client_data, epochs=1):
    # get x and y from client_data
    x = [data[0] for data in client_data]  #image data
    y = [data[1] for data in client_data]  #label data

    # Convert to numpy arrays for model fitting
    x = np.array(x)
    y = np.array(y)

    model.fit(x, y, epochs=epochs, verbose=0)  # train the model
    return model.get_weights()  # Return the new model weights

global_model = create_model()
global_weights = global_model.get_weights()
round = 0;
while round < 5:  # number of rounds
    client_weights = []

    for client in client_data:
        local_model = create_model()
        local_model.set_weights(global_weights)
        client_weights.append(client_update(local_model, client, epochs=1))

    global_weights = agg_weights(client_weights)
    global_model.set_weights(global_weights)
    round += 1
    print(f"Round {round} complete")

test_loss, test_acc = global_model.evaluate(x_train, y_train, verbose=2)
print("Global Model Accuracy: " + str(test_acc))

Round 1 complete
Round 2 complete
Round 3 complete
Round 4 complete
Round 5 complete
Round 6 complete
1875/1875 - 3s - loss: 0.4391 - accuracy: 0.8855 - 3s/epoch - 2ms/step
Global Model Accuracy: 0.8855166435241699
